In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../../data/data_taxi_central_based.parquet")

In [3]:
first_rows = df.groupby('TRIP_ID').first().reset_index()

In [4]:
df = df[(df['POLYLINE_LENGTH'] >= 30) & (df['POLYLINE_LENGTH'] < 276)]

In [5]:
import timesfm

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)].


In [6]:
df_lon = df.drop(columns=['POLYLINE_LENGTH', 'LAT'])
df_lat = df.drop(columns=['POLYLINE_LENGTH', 'LON'])

In [7]:
tfm_lat = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cpu",
          per_core_batch_size=32,
          horizon_len=20,
          num_layers=50,
          use_positional_embedding=False,
          context_len=256,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
tfm_lon = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=20,
          num_layers=50,
          use_positional_embedding=False,
          context_len=256,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
df_lat = df_lat.sort_values(by=['TRIP_ID', 'TIMESTAMP'])

In [10]:
df_lon = df_lon.sort_values(by=['TRIP_ID', 'TIMESTAMP'])

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler_lat = MinMaxScaler()
df_lat['LAT'] = scaler_lat.fit_transform(df_lat[['LAT']])

scaler_lon = MinMaxScaler()
df_lon['LON'] = scaler_lon.fit_transform(df_lon[['LON']])

In [12]:
df_lat['TIMESTAMP'] = pd.to_datetime(df_lat['TIMESTAMP'], unit='s')
df_lon['TIMESTAMP'] = pd.to_datetime(df_lon['TIMESTAMP'], unit='s')

In [13]:
df_lon = df_lon.rename(columns={'TIMESTAMP': 'ds', 'LON': 'y', 'TRIP_ID': 'unique_id', 'ORIGIN_CALL': 'origin_call', 'TAXI_ID': 'taxi_id'})
df_lat = df_lat.rename(columns={'TIMESTAMP': 'ds', 'LAT': 'y', 'TRIP_ID': 'unique_id', 'ORIGIN_CALL': 'origin_call', 'TAXI_ID': 'taxi_id'})

In [14]:
from collections import defaultdict


def get_taxi_data_fn(df, batch_size: int = 256, horizon_len: int = 20):
    examples = defaultdict(list)
    num_examples = 0

    for ride_id, sub_df in df.groupby("unique_id"):
        if len(sub_df) <= horizon_len:
            continue

        context_df = sub_df.iloc[:-horizon_len]
        horizon_df = sub_df.iloc[-horizon_len:]

        num_examples += 1
        examples["unique_id"].append(ride_id)
        examples["origin_call"].append(sub_df["origin_call"].iloc[0])
        examples["taxi_id"].append(sub_df["taxi_id"].iloc[0])

        examples["inputs"].append(context_df["y"].tolist())
        examples["outputs"].append(horizon_df["y"].tolist())
        examples["ds"].append(sub_df["ds"].tolist())

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {
                k: v[i * batch_size : (i + 1) * batch_size]
                for k, v in examples.items()
            }

    return data_fn

In [15]:
import time

input_data = get_taxi_data_fn(df_lat)
forecast_df_lat = []

for i, example in enumerate(input_data()):
  start_time = time.time()
  cov_forecast, _ = tfm_lat.forecast_with_covariates(
      inputs=example["inputs"],
      static_numerical_covariates={
          "origin_call": example["origin_call"],
          "taxi_id": example["taxi_id"],
      },
      freq=[0] * len(example["inputs"]),
      xreg_mode="xreg + timesfm",
      ridge=0.0,
      force_on_cpu=False,
      normalize_xreg_target_per_input=True,
  )
  print(
      f"\rFinished batch {i} linear in {time.time() - start_time} seconds",
      end="",
  )
  forecast_df_lat.append(cov_forecast)

print()

Finished batch 1114 linear in 2.6791915893554688 seconds


In [16]:
import time

input_data = get_taxi_data_fn(df_lon)
forecast_df_lon = []

for i, example in enumerate(input_data()):
  start_time = time.time()
  cov_forecast, _ = tfm_lon.forecast_with_covariates(
      inputs=example["inputs"],
      static_numerical_covariates={
          "origin_call": example["origin_call"],
          "taxi_id": example["taxi_id"],
      },
      freq=[0] * len(example["inputs"]),
      xreg_mode="xreg + timesfm",
      ridge=0.0,
      force_on_cpu=False,
      normalize_xreg_target_per_input=True,
  )
  print(
      f"\rFinished batch {i} linear in {time.time() - start_time} seconds",
      end="",
  )
  forecast_df_lon.append(cov_forecast)

print()

Finished batch 1114 linear in 1.8832180500030518 seconds


In [17]:
def forecasts_to_df(forecasts, examples, coord_name):
    records = []
    for arr, uid, call, taxi, coor in zip(forecasts,
                                    examples["unique_id"],
                                    examples["origin_call"],
                                    examples["taxi_id"],
                                    examples["inputs"]):
        for step, value in enumerate(arr):
            records.append({
                "unique_id": uid,
                "origin_call": call,
                "taxi_id": taxi,
                "horizon_step": step + 1,
                f"yhat_{coord_name}": value,
            })
    return pd.DataFrame(records)

In [18]:
input_data = get_taxi_data_fn(df_lat)
df_lat_forecast = []
for example, forecast in zip(input_data(), forecast_df_lat):
    df_lat = forecasts_to_df(forecast, example, coord_name="lat")
    df_lat_forecast.append(df_lat)
df_lat_forecast = pd.concat(df_lat_forecast, ignore_index=True)
df_lat_forecast

,unique_id,origin_call,taxi_id,horizon_step,yhat_lat
0,1372637254620000657,39233.0,20000657,1,0.585462
1,1372637254620000657,39233.0,20000657,2,0.585471
2,1372637254620000657,39233.0,20000657,3,0.585486
3,1372637254620000657,39233.0,20000657,4,0.585513
4,1372637254620000657,39233.0,20000657,5,0.585509
...,...,...,...,...,...
5704575,1404172319620000571,4650.0,20000571,16,0.575430
5704576,1404172319620000571,4650.0,20000571,17,0.575442
5704577,1404172319620000571,4650.0,20000571,18,0.575414
5704578,1404172319620000571,4650.0,20000571,19,0.575407


In [19]:
input_data = get_taxi_data_fn(df_lon)
df_lon_forecast = []
for example, forecast in zip(input_data(), forecast_df_lon):
    df_lon = forecasts_to_df(forecast, example, coord_name="lon")
    df_lon_forecast.append(df_lon)
df_lon_forecast = pd.concat(df_lon_forecast, ignore_index=True)
df_lon_forecast

,unique_id,origin_call,taxi_id,horizon_step,yhat_lon
0,1372637254620000657,39233.0,20000657,1,0.638454
1,1372637254620000657,39233.0,20000657,2,0.638667
2,1372637254620000657,39233.0,20000657,3,0.638778
3,1372637254620000657,39233.0,20000657,4,0.638917
4,1372637254620000657,39233.0,20000657,5,0.639057
...,...,...,...,...,...
5704575,1404172319620000571,4650.0,20000571,16,0.640514
5704576,1404172319620000571,4650.0,20000571,17,0.640549
5704577,1404172319620000571,4650.0,20000571,18,0.640562
5704578,1404172319620000571,4650.0,20000571,19,0.640543


In [20]:
df_lat_forecast["yhat_lat"] = scaler_lat.inverse_transform(df_lat_forecast[["yhat_lat"]])
df_lon_forecast["yhat_lon"] = scaler_lon.inverse_transform(df_lon_forecast[["yhat_lon"]])
predictions = pd.merge(df_lat_forecast, df_lon_forecast, on=['unique_id', 'origin_call', 'taxi_id', 'horizon_step'], how='left')
predictions

,unique_id,origin_call,taxi_id,horizon_step,yhat_lat,yhat_lon
0,1372637254620000657,39233.0,20000657,1,41.180661,-8.628694
1,1372637254620000657,39233.0,20000657,2,41.180692,-8.627177
2,1372637254620000657,39233.0,20000657,3,41.180740,-8.626384
3,1372637254620000657,39233.0,20000657,4,41.180828,-8.625398
4,1372637254620000657,39233.0,20000657,5,41.180816,-8.624404
...,...,...,...,...,...,...
5704575,1404172319620000571,4650.0,20000571,16,41.147688,-8.614034
5704576,1404172319620000571,4650.0,20000571,17,41.147728,-8.613786
5704577,1404172319620000571,4650.0,20000571,18,41.147634,-8.613692
5704578,1404172319620000571,4650.0,20000571,19,41.147612,-8.613827


In [21]:
df_lat = df_lat.rename(columns={"y": "lat"})
df_lon = df_lon.rename(columns={"y": "lon"})

In [22]:
df_lon_test = df_lon.groupby('unique_id', group_keys=False).tail(1)
df_lat_test = df_lat.groupby('unique_id', group_keys=False).tail(1)
test = pd.merge(df_lat_test, df_lon_test, on=['unique_id', 'origin_call', 'taxi_id', 'ds'], how='left')
test

KeyError: 'ds'

In [ ]:
test["lat"] = scaler_lat.inverse_transform(test[["lat"]])
test["lon"] = scaler_lon.inverse_transform(test[["lon"]])
test

In [ ]:
predictions = predictions.groupby('unique_id', group_keys=False).tail(1)
predictions

In [ ]:
predictions = predictions.drop(['origin_call', 'taxi_id', 'horizon_step'], axis=1)
test = test.drop(['origin_call', 'taxi_id', 'ds'], axis=1)
df = pd.merge(test, predictions, on='unique_id', how='left')
df

In [ ]:
from evaluation_script import haversine_distance
df['distance_delta'] = df.apply(lambda row: haversine_distance(
    row['lat'], row['lon'], row['yhat_lat'], row['yhat_lon']), axis=1)
df['distance_delta'].mean()

In [ ]:
df['distance_delta'].median()

In [ ]:
df['distance_delta']

In [ ]:
df.to_parquet("results.parquet")